In [2]:
from legisph.senate.website import SenateWebsite
senate = SenateWebsite()
bills = list(filter(lambda b: not isinstance(b, Exception), senate.fetch_all_bills()))

In [3]:
import polars as pl
from itertools import chain

all_statuses = [
    *[b.legislative_status for b in bills],
    *list(chain(*[b.legislative_history for b in bills])),
]
[
    s.item.split("\n")
    for s in all_statuses 
    if s.item.find("\n") > 0
]

[['Letter from Senate Secretary Myra Marie D. Villarica, transmitting nine (9) enrolled',
  'copies of the consolidated version of SBN-2522 and HBN-6522, for the signature of Speaker Lord Allan Jay',
  'Q. Velasco and Secretary General Mark Llandro L. Mendoza;'],
 ['Enrolled copies of SBN-2506 and HBN-9459 sent to the Office of the',
  'President through PLLO for the signature of the President of the Philippines, His Excellency, President',
  'Rodrigo Roa Duterte;'],
 ['Letter from Senate Secretary Myra Marie D. Villarica, transmitting nine (9) enrolled',
  'copies of SBN-2504 and HBN-10015 for the signature of Speaker Lord Allan Jay Q. Velasco and Secretary General Mark',
  'Llandro L. Mendoza;'],
 ['Enrolled copies of SBN-2504 and HBN-10015 sent to the Office of the',
  'President through PLLO for the signature of the President of the Philippines, His Excellency, President',
  'Rodrigo Roa Duterte;'],
 ['Letter from Senate Secretary Myra Marie D. Villarica, dated June 21, 2022, trans

In [62]:
from legisph.senate.bill_status import * 

unparsed_statuses = [
    status
    for status in all_statuses
    if len(
        parse_senate_bill_status(
            status,
            classes=[
                Introduced,
                FirstReading,
                OrganizationalBriefing,
                PendingInCommittee,
                CommitteeReportCalendaredForOrdinaryBusiness,
                ConsolidatedOrSubstitutedInCommitteeReport,
                TechnicalWorkingGroup,
                CommitteeProceedings,
                SubmittedCommitteeReport,
                PendingSecondReading,
                ApprovedOnSecondReading,
                TransferredToCalendarForSpecialOrder,
                PeriodOfInterpellationClosed,
                SponsorshipSpeech,
                SponsorshipSpeechInsertedIntoRecord,
                ApprovedOnThirdReading,
                SentToHouseOfRepresentatives,
                IndividualAmendmentsOpened,
                IndividualAmendmentsClosed,
                CommitteeAmendmentsOpened,
                CommitteeAmendmentsClosed,
                CopiesDistributed,
                ConsolidatedWithApprovedBill,
                ApprovedByPresident,
                ApprovedConferenceCommitteeReport,
                Interpellation,
                PendingInHouseOfRepresentatives,
                Withdrawn,
                Archived,
                Sponsored
            ],
        )
    )
    == 0
]
status_df = pl.DataFrame({"status": [status.item for status in unparsed_statuses]})
list(
    (
        status_df.groupby("status")
            .agg([pl.count()])
            .sort("count", reverse=True)
            .filter(pl.col("count") > 50)
    )["status"]
)
# (
#     status_df.groupby("status")
#             .agg([pl.count()])
#             .sort("count", reverse=True)
#             .filter(pl.col("count") > 10)
# )

['Abstention:  (None) ;',
 'Against:  (None) ;',
 'Against:         N o n e;',
 'Abstention:    N o n e;',
 'Abstention:   N o n e;',
 'Against:  N o n e ;',
 'Abstention:  N o n e ;',
 'Motion of the Majority Leader, Senator Alan Peter Compañero S. Cayetano, that the matters appearing in the Reference of Business for todays session be deemed read and transmitted to the Archives;']

In [59]:
from legisph.senate.bill_status import * 
from legisph.senate.models import SenateBill
import datetime
matches = set([
    s.item
    for s in all_statuses
    if s.item.lower().find("sponsor:") > -1
])
matches
# unparsed_statuses = [
#     status
#     for status in sponsorship_speech
#     if SponsorshipSpeech.parse(SenateBill.SenateBillStatus(
#         date = datetime.date.today(),
#         item = status
#     ))[0] is None
# ]
# unparsed_statuses

{'Calendared for Ordinary Business;\nSponsor: Senator Enrile;',
 'Sponsor:  Senator Companera Pia S. Cayetano;',
 'Sponsor:  Senator Gordon;',
 'Sponsor: Sen. Angara',
 'Sponsor: Senator "COMPANERA" PIA S. CAYETANO;',
 'Sponsor: Senator "Companera" Pia S. Cayetano;',
 'Sponsor: Senator ALAN PETER COMPANERO S. CAYETANO;',
 'Sponsor: Senator ALFREDO S. LIM;',
 'Sponsor: Senator ANTONIO "SONNY" F. TRILLANES;',
 'Sponsor: Senator AQUILINO "KOKO" III PIMENTEL;',
 'Sponsor: Senator AQUILINO KOKO III L. PIMENTEL;',
 'Sponsor: Senator Alan Peter Companero S. Cayetano;',
 'Sponsor: Senator Biazon;',
 'Sponsor: Senator CHRISTOPHER LAWRENCE T. GO;',
 'Sponsor: Senator COMPANERA PIA S. CAYETANO;',
 'Sponsor: Senator CYNTHIA A. VILLAR;',
 'Sponsor: Senator EDGARDO J. ANGARA;',
 'Sponsor: Senator EMMANUEL "MANNY" D. PACQUIAO;',
 'Sponsor: Senator ESCUDERO;',
 'Sponsor: Senator FERDINAND "BONGBONG" R. MARCOS;',
 'Sponsor: Senator FRANCIS "CHIZ" G. ESCUDERO ;',
 'Sponsor: Senator FRANCIS "CHIZ" G. ESC

In [6]:
a = list(filter(
    lambda b: "Consolidated/Substituted in the Committee Report" == b.legislative_status.item,
    bills
))
a[0]

SenateBill(
    url="http://legacy.senate.gov.ph/lis/bill_res.aspx?q=SBN-1238&congress=19",
    congress=19,
    billno="SBN-1238",
    congress_text="19th Congress",
    billno_text="Senate Bill No. 1238",
    title="PREPAID SIM CARD REGISTRATION ACT",
    long_title="AN ACT PROHIBITING TEXT SCAMS, MISLEADING ADVERTISEMENTS, AND FRAUDULENT SALES PROMOTIONS, MANDATING FOR THIS PURPOSE THE REGISTRATION OF ALL USERS OF SUBSCRIBER IDENTITY MODULE (SIM)\nCARDS, AND PROVIDING PENALTIES FOR THE VIOLATIONS THEREOF",
    filing_date=datetime.date(2022, 8, 31),
    filers=[Senator(name="Villar, Cynthia A.")],
    links=[
        Link(
            url="http://legacy.senate.gov.ph/lisdata/3923835675!.pdf",
            name="SBN-1238 (as filed)",
        )
    ],
    scope="National",
    legislative_status=SenateBillStatus(
        date=datetime.date(2022, 9, 13),
        item="Consolidated/Substituted in the Committee Report",
    ),
    subjects=[
        Subject(name="Prepaid Calling Cards/Sim

In [50]:
a = list(filter(
    lambda b: 'Sponsor: Senator FRANCIS "CHIZ" G. ESCUDERO;' in [h.item for h in b.legislative_history],
    bills
))

In [51]:
a[17].legislative_history

[SenateBillStatus(date=datetime.date(2016, 8, 25), item='Introduced by Senator JOSEPH VICTOR G. EJERCITO;'),
 SenateBillStatus(date=datetime.date(2016, 8, 30), item='Read on First Reading and Referred to the Committees on EDUCATION, ARTS AND CULTURE; and FINANCE;'),
 SenateBillStatus(date=datetime.date(2017, 5, 23), item='Conducted JOINT COMMITTEE MEETINGS/HEARINGS;'),
 SenateBillStatus(date=datetime.date(2017, 7, 27), item='Returned and submitted jointly by the Committees on EDUCATION, ARTS AND CULTURE and FINANCE per Committee Report No. 131, recommending its approval without amendment;'),
 SenateBillStatus(date=datetime.date(2017, 7, 31), item='Committee Report Calendared for Ordinary Business;'),
 SenateBillStatus(date=datetime.date(2017, 7, 31), item='Sponsor: Senator FRANCIS "CHIZ" G. ESCUDERO;'),
 SenateBillStatus(date=datetime.date(2017, 8, 14), item='Transferred from the Calendar for Ordinary Business to the Calendar for Special Order;'),
 SenateBillStatus(date=datetime.date(2